# Limpieza de Datos 

## Colonias prioritarias CDMX

El archivo ```colonias_prioritarias.csv``` contiene una lista de las 200 colonias de la Ciudad de México con mayor prioridad de atención médica según el gobierno de la ciudad. Este archivo fue extraido de: https://datos.cdmx.gob.mx/dataset/colonias-de-atencion-prioritaria-covid-kioscos

In [275]:
import pandas as pd
data_path = '../../data/'

In [276]:
colonias_df = pd.read_csv(data_path + 'colonias_prioritarias.csv')
colonias_df.columns = colonias_df.columns.str.lower()
colonias_df.columns = colonias_df.columns.str.replace(' ', '_')
colonias_df.drop(columns=['tasa_activos_(100mil_habitantes)','geo_shape', 'clave_colonia', 'centroide_colonia'], inplace=True)
colonias_df.rename(columns={'alcaldía': 'alcaldia', 'población_total':'poblacion_total'}, inplace=True)
colonias_df.colonia = colonias_df.colonia.str.lower()
colonias_df.alcaldia = colonias_df.alcaldia.str.lower()
colonias_df.porcentaje_activos = colonias_df.porcentaje_activos*100

In [277]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
0,alvaro obregon,barrio norte,57,14367.414210,0.2562
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753
3,alvaro obregon,merced gomez,48,6992.834615,0.2157
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663


### Identificación y corrección de valores nulos

In [278]:
colonias_df.isna().sum()

alcaldia              0
colonia               0
casos_activos         0
poblacion_total       2
porcentaje_activos    0
dtype: int64

In [279]:
colonias_df[colonias_df.poblacion_total.isna()]

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
47,tlahuac,la conchita i,22,NaN,0.0989
111,tlahuac,la conchita ii,102,NaN,0.4584


In [280]:
colonias_df.loc[47, 'poblacion_total'] = (colonias_df.loc[47].casos_activos * 100) / colonias_df.loc[47].porcentaje_activos
colonias_df.loc[111, 'poblacion_total'] = (colonias_df.loc[111].casos_activos * 100) / colonias_df.loc[111].porcentaje_activos

In [281]:
colonias_df.isna().sum()

alcaldia              0
colonia               0
casos_activos         0
poblacion_total       0
porcentaje_activos    0
dtype: int64

## Guardar dataframe limpio

In [282]:
colonias_df.to_csv(data_path + 'colonias_prioritarias_clean.csv', index=False)

In [283]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
0,alvaro obregon,barrio norte,57,14367.414210,0.2562
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753
3,alvaro obregon,merced gomez,48,6992.834615,0.2157
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663


## Negocios CDMX
Esta lista contiene los negocios dentro de la Ciudad de México que solicitaron financiamiento dentro del programa: Financiamiento para Microempresas del Fondo para el Desarrollo Social de la Ciudad de México. 

La base de datos original se encuentra en: https://datos.cdmx.gob.mx/dataset/solicitudes-financiamiento-microempresas

In [284]:
negocios_df = pd.read_csv(data_path + 'microempresas.csv')

In [285]:
def lower_cols_rows(df):
    df.columns = df.columns.str.lower()
    for column in df.columns:
        df[str(column)] = df[str(column)].str.lower()
    return df

In [286]:
negocios_df.drop(columns=['SEXO', 'FOLIO', 'ESTADO_DE_NACIMIENTO', 'FECHA_INICIO_SOLICITUD', 'FECHA_TERMINO'], inplace=True)
negocios_df = lower_cols_rows(negocios_df)

In [287]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados,estatus
0,joyeria�,alvaro obregon,1260,la mexicana,joyeria�,0,cobrado
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3,cobrado
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0,cobrado
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3,cobrado
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0,cobrado


In [288]:
negocios_df = negocios_df[negocios_df.estatus == 'cobrado']
negocios_df.drop(columns=['estatus'], inplace=True)
negocios_df.shape

(51198, 6)

In [289]:
negocios_df.isna().sum()

ocupacion          9
alcaldia           0
c_p                1
colonia           11
giro             128
num_empleados      0
dtype: int64

### Ocupación

In [290]:
negocios_df[negocios_df.ocupacion.isna()]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
117391,NaN,venustiano carranza,15290,10 de mayo,juegos mecanicos feria,1
118507,NaN,cuauhtemoc,6220,peralvillo,panaderia,1
118524,NaN,azcapotzalco,2630,coltongo,purificadora de agua,1
118532,NaN,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118537,NaN,iztacalco,8910,barrio santa cruz,artesanias y juegos didacticos,1
118542,NaN,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118908,NaN,cuauhtemoc,6720,doctores,"promotora deportiva gama soccer, ac",1
118923,NaN,iztapalapa,9099,heroes de churubusco,liga de futbol siglo xxi plus a.c,1
118924,NaN,iztapalapa,9060,escuadron 201,escuela de futbol pumas 201,1


In [291]:
indices = list(negocios_df[negocios_df.ocupacion.isna()].index)
for index in indices:
    negocios_df.loc[index, 'ocupacion'] = negocios_df.loc[index, 'giro']

In [292]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
117391,juegos mecanicos feria,venustiano carranza,15290,10 de mayo,juegos mecanicos feria,1
118507,panaderia,cuauhtemoc,6220,peralvillo,panaderia,1
118524,purificadora de agua,azcapotzalco,2630,coltongo,purificadora de agua,1
118532,comercio,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118537,artesanias y juegos didacticos,iztacalco,8910,barrio santa cruz,artesanias y juegos didacticos,1
118542,comercio,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118908,"promotora deportiva gama soccer, ac",cuauhtemoc,6720,doctores,"promotora deportiva gama soccer, ac",1
118923,liga de futbol siglo xxi plus a.c,iztapalapa,9099,heroes de churubusco,liga de futbol siglo xxi plus a.c,1
118924,escuela de futbol pumas 201,iztapalapa,9060,escuadron 201,escuela de futbol pumas 201,1


In [293]:
negocios_df.isna().sum()

ocupacion          0
alcaldia           0
c_p                1
colonia           11
giro             128
num_empleados      0
dtype: int64

### Colonia

In [294]:
negocios_df[negocios_df.colonia.isna()]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
38833,comerciante,coyoacan,4470,NaN,tienda de abarrotes,2
38950,comerciante,coyoacan,4330,NaN,ferreteria,4
38991,comerciante,tlahuac,13300,NaN,cocina,1
40360,comerciante,venustiano carranza,15290,NaN,alimentos,2
40361,comerciante,tlalpan,14738,NaN,renta de equipo de computo,1
118074,comerciante,xochimilco,16090,NaN,cocina,3
118123,comerciante,tlahuac,13200,NaN,"venta de fragancias, escncias y perfumes",1
118195,comerciante,tlalpan,14430,NaN,cafeteria,1
118203,comerciante,coyoacan,4480,NaN,venta de ropa y accesorios,1
118289,comerciante,xochimilco,16090,NaN,panaderia,2


In [295]:
indices = list(negocios_df[negocios_df.colonia.isna()].index)

In [296]:
negocios_df.loc[38833, 'colonia'] = 'Presidentes Ejidales 1a Seccion'
negocios_df.loc[38950, 'colonia'] = 'El Rosedal'
negocios_df.loc[38991, 'colonia'] = 'Santa Ana Centro'
negocios_df.loc[40360, 'colonia'] = '10 de Mayo '
negocios_df.loc[40361, 'colonia'] = 'Bosques del Pedregal'
negocios_df.loc[118074, 'colonia'] = 'Tablas de San Lorenzo'
negocios_df.loc[118123, 'colonia'] = 'Miguel Hidalgo'
negocios_df.loc[118195, 'colonia'] = 'Tlalcoligia'
negocios_df.loc[118203, 'colonia'] = 'Culhuacan CTM CROC'
negocios_df.loc[118289, 'colonia'] = 'Tablas de San Lorenzo'
negocios_df.loc[118585, 'colonia'] = 'El Rosario'

In [297]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
38833,comerciante,coyoacan,4470,Presidentes Ejidales 1a Seccion,tienda de abarrotes,2
38950,comerciante,coyoacan,4330,El Rosedal,ferreteria,4
38991,comerciante,tlahuac,13300,Santa Ana Centro,cocina,1
40360,comerciante,venustiano carranza,15290,10 de Mayo,alimentos,2
40361,comerciante,tlalpan,14738,Bosques del Pedregal,renta de equipo de computo,1
118074,comerciante,xochimilco,16090,Tablas de San Lorenzo,cocina,3
118123,comerciante,tlahuac,13200,Miguel Hidalgo,"venta de fragancias, escncias y perfumes",1
118195,comerciante,tlalpan,14430,Tlalcoligia,cafeteria,1
118203,comerciante,coyoacan,4480,Culhuacan CTM CROC,venta de ropa y accesorios,1
118289,comerciante,xochimilco,16090,Tablas de San Lorenzo,panaderia,2


In [298]:
negocios_df.isna().sum()

ocupacion          0
alcaldia           0
c_p                1
colonia            0
giro             128
num_empleados      0
dtype: int64

### Giro

In [299]:
indices = list(negocios_df[negocios_df.giro.isna()].index)
for index in indices:
    negocios_df.loc[index, 'giro'] = negocios_df.loc[index, 'ocupacion']

In [300]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
8676,autoempleda,iztapalapa,9230,jos maria morelos y pavon,autoempleda,1
10342,comerciante,azcapotzalco,2400,san juan tlihuaca,comerciante,2
10349,comerciante,iztapalapa,9840,los reyes culhuacn,comerciante,1
10505,comerciante,alvaro obregon,1830,santa rosa xochiac,comerciante,2
11029,comerciante,cuauhtemoc,6860,vista alegre,comerciante,2
...,...,...,...,...,...,...
86127,restaurantero,iztapalapa,9230,jos maria morelos y pavon,restaurantero,3
86174,estilista,tlahuac,13430,ampliacion selene,estilista,3
86391,comerciante,xochimilco,16038,ampliacion san marcos norte,comerciante,1
87050,"albañil , pintor y plomero",iztacalco,8710,tlazintla,"albañil , pintor y plomero",2


In [301]:
negocios_df.isna().sum()

ocupacion        0
alcaldia         0
c_p              1
colonia          0
giro             0
num_empleados    0
dtype: int64

In [302]:
#negocios_df.drop(columns=['c_p'], inplace=True)

## Limpieza de cadenas

### Colonias

In [311]:
negocios_df[negocios_df['c_p'] == str(4369)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
500,panaderia,coyoacan,4369,pedregal de santo domingo,panaderia,2
501,panaderia,coyoacan,4369,pedregal de santo domingo,panaderia,2
502,panaderia,coyoacan,4369,pedregal de santo domingo,panaderia,1
503,panaderia,coyoacan,4369,pedregal de santo domingo,panaderia,1
519,cocinas economicas�,coyoacan,4369,santo domingo,cocinas economicas�,0
...,...,...,...,...,...,...
117838,comerciante,coyoacan,4369,pedregal de santo domingo,flores,1
117942,locatario publico,coyoacan,4369,santo domingo,estetica,1
117943,locatario publico,coyoacan,4369,santo domingo,venta de articulos para mascotas,1
118263,comerciante,coyoacan,4369,pedregal de santo domingo,venta de alimentos,1


## Guardar dataframe limpio

In [304]:
negocios_df.to_csv(data_path + 'microempresas_clean.csv', index=False)

In [305]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
0,joyeria�,alvaro obregon,1260,la mexicana,joyeria�,0
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
